In [2]:
import os 
import sys
import numpy as np 
import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numba
import json
import sklearn
import glob

In [4]:
csvs = []
for n in glob.glob('../data/dwd_weather/*Helgoland.csv'):
    
    s = pd.read_csv(n,index_col=False,skiprows=1,names=["Produkt_Code","SDO_ID","Zeitstempel","Wert","Qualitaet_Byte","Qualitaet_Niveau"])
    n = n.replace("../data/dwd_weather/","").replace("_Helgoland.csv","")
    s.index = pd.to_datetime(s.Zeitstempel,format= "%Y-%m-%d")
    s = s['Wert'].to_frame()
    s.columns = [n]
    csvs.append(s)
    
df = pd.concat(csvs,axis= 1,join = 'outer')
df['year'] = df.index.year
df = df.loc[df.year < 2017]
df.columns

Index(['precipitation_form', 'temp_min', 'snow_depth', 'sunshine_duration',
       'air_pressure_mean', 'wind_spead_mean', 'relative_humidity',
       'air_temp_min', 'mean_cloud_coverage', 'temp_max',
       'precipitation_height', 'wind_max', 'vapor_pressure_mean', 'temp_mean',
       'year'],
      dtype='object')

In [5]:
df = df.interpolate('time')
df.describe().T

,count,mean,std,min,25%,50%,75%,max
precipitation_form,20820.0,3.282805,3.045951,0.0,0.0,1.0,6.0,8.0
temp_min,20820.0,7.817550,5.696681,-10.7,3.5,7.7,12.8,21.1
snow_depth,20820.0,0.100768,0.817768,0.0,0.0,0.0,0.0,25.0
sunshine_duration,20820.0,4.720283,4.486994,0.0,0.4,3.6,8.0,16.6
air_pressure_mean,20820.0,1013.402317,10.372580,959.4,1007.2,1014.2,1020.3,1047.9
wind_spead_mean,20820.0,7.721446,3.250186,0.8,5.2,7.3,9.8,20.8
relative_humidity,20820.0,83.067047,8.481141,40.0,78.0,84.0,89.0,100.0
air_temp_min,20820.0,6.710495,5.665077,-13.6,2.5,6.7,11.5,20.6
mean_cloud_coverage,20820.0,5.356686,2.110551,0.0,4.0,5.7,7.1,8.0
temp_max,20820.0,11.215312,5.985500,-8.2,6.5,11.1,16.2,28.7


normalised = (df - df.mean()) / df.std()
normalised.to_csv("../data/daily_weather_normalised.csv")
df.to_csv("../data/daily_weather_df.csv")

In [6]:
weather = (df - df.mean()) / df.std()


In [8]:

submission_ds = pd.read_csv('../data/bbdc_student/bbdc_2023_AWI_data_evaluate_skeleton_student.csv', delimiter=';',skiprows=[1],index_col=0)
submission_ds['datum'] =submission_ds.index
submission_ds.index = pd.to_datetime(submission_ds.index + " " + submission_ds.Uhrzeit,format='%d.%m.%Y %H:%M')
submission_ds["time"] = submission_ds.index
submission_ds['time_as_number'] = submission_ds.index.hour + submission_ds.index.minute/60

df = pd.read_csv('../data/trainings_df.csv',index_col=0)
df.pop('missing_values')
df['missing_values'] = df.isna().sum(axis=1)
df.head()
df['datum'] = df.index
df.index = pd.to_datetime(df.index + " " + df.Uhrzeit,format='%d.%m.%Y %H:%M')
df["time"] = df.index
df['time_as_number'] = df.index.hour + df.index.minute/60
df.head(1)

times = pd.concat((df[['time_as_number','datum','time']], submission_ds[['time_as_number','datum','time']])).set_index('datum')
daily_index = pd.to_datetime(times.index,format='%d.%m.%Y')
weather.loc[daily_index,'time_as_number'] = times.time_as_number
times.to_csv('../data/times.csv')

In [10]:

submission_ds = pd.read_csv('../data/bbdc_prof/bbdc_2023_AWI_data_evaluate_skeleton_professional.csv', delimiter=';',skiprows=[1],index_col=0)
submission_ds['datum'] =submission_ds.index
submission_ds.index = pd.to_datetime(submission_ds.index + " " + submission_ds.Uhrzeit,format='%d.%m.%Y %H:%M')
submission_ds["time"] = submission_ds.index
submission_ds['time_as_number'] = submission_ds.index.hour + submission_ds.index.minute/60

df = pd.read_csv('../data/trainings_df.csv',index_col=0)
df.pop('missing_values')
df['missing_values'] = df.isna().sum(axis=1)
df.head()
df['datum'] = df.index
df.index = pd.to_datetime(df.index + " " + df.Uhrzeit,format='%d.%m.%Y %H:%M')
df["time"] = df.index
df['time_as_number'] = df.index.hour + df.index.minute/60
df.head(1)

times = pd.concat((df[['time_as_number','datum','time']], submission_ds[['time_as_number','datum','time']])).set_index('datum')
daily_index = pd.to_datetime(times.index,format='%d.%m.%Y')
weather.loc[daily_index,'time_as_number'] = times.time_as_number
times.to_csv('../data/prof_times.csv')

In [11]:
weather

,precipitation_form,temp_min,snow_depth,sunshine_duration,air_pressure_mean,wind_spead_mean,relative_humidity,air_temp_min,mean_cloud_coverage,temp_max,precipitation_height,wind_max,vapor_pressure_mean,temp_mean,year,time_as_number
Zeitstempel,,,,,,,,,,,,,,,,
1960-01-01,-0.749456,-0.213730,-0.123224,-1.051992,-1.253528,0.147239,1.053273,-0.143069,0.304809,-0.470355,0.143169,0.167425,-0.254791,-0.370352,-1.701822,NaN
1960-01-02,-1.077760,-0.319054,-0.123224,-1.051992,-0.848614,-1.360367,1.760725,-0.090113,1.252428,-0.637426,-0.487211,-1.423625,-0.254791,-0.561974,-1.701822,NaN
1960-01-03,-0.749456,-0.529703,-0.123224,-1.051992,0.289001,-1.514204,1.878633,-0.196025,1.252428,-0.771082,-0.390230,-1.312622,-0.358423,-0.718755,-1.701822,NaN
1960-01-04,-0.749456,-0.775460,-0.123224,-1.051992,1.243440,-0.345040,1.760725,-0.584369,1.252428,-0.854617,-0.390230,-0.498596,-0.513871,-0.753596,-1.701822,NaN
1960-01-05,-0.749456,-0.354162,-0.123224,-1.051992,0.423972,0.854891,0.699547,-0.213677,0.304809,-0.520477,-0.365984,0.944450,-0.358423,-0.422612,-1.701822,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,0.235459,-0.038189,-0.123224,-0.829126,1.879733,-0.116482,-0.479540,-0.125417,0.589094,-0.353406,-0.462966,-0.630156,-0.436147,-0.178730,1.701822,NaN
2016-12-28,-1.077760,-0.512149,-0.123224,-1.051992,2.679920,-0.230761,-0.479540,-0.390197,0.446952,-0.553891,-0.487211,-0.712381,-0.643411,-0.405192,1.701822,NaN
2016-12-29,-1.077760,-0.915893,-0.123224,-0.784553,2.429259,-0.345040,0.581638,-0.725585,-0.453287,-1.021688,-0.487211,-0.794605,-0.876582,-0.980058,1.701822,NaN


In [12]:

def get_year_encodings(df):


    date_time = df.index.to_series()
    timestamp_s = date_time.map(pd.Timestamp.timestamp)
    day = 24*60*60
    year = (365.2425)*day

    df['Year_sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df['Year_cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df['Weekday_sin'] = np.sin(df.index.weekday * 2 * np.pi/ 7)
    df['Weekday_cos'] = np.cos(df.index.weekday * 2 * np.pi/ 7)
    return df


weather = get_year_encodings(weather)

In [13]:
weather.year = weather.index.year
weather = (weather - weather.mean()) / weather.std()
(weather.index.weekday > 4).sum()

5949

In [14]:
weather = weather.loc[~(weather.index.weekday > 4)]

In [15]:
(df.index.weekday > 4).sum()

0

In [16]:
weather.describe().T

,count,mean,std,min,25%,50%,75%,max
precipitation_form,14871.0,1.664747e-03,1.001041,-1.077760,-1.077760,-7.494557e-01,0.892068,1.548677
temp_min,14871.0,1.374102e-03,0.999278,-3.145261,-0.757906,-2.063490e-02,0.874623,2.314058
snow_depth,14871.0,-2.592564e-03,0.992635,-0.123224,-0.123224,-1.232238e-01,-0.123224,30.447798
sunshine_duration,14871.0,-5.010546e-03,1.000231,-1.051992,-0.985132,-2.496735e-01,0.730938,2.625302
air_pressure_mean,14871.0,-5.077444e-03,1.006166,-4.222895,-0.607594,7.690309e-02,0.655351,3.084834
wind_spead_mean,14871.0,6.561890e-03,1.005211,-2.129554,-0.775785,-1.296682e-01,0.639519,3.993173
relative_humidity,14871.0,2.610094e-03,0.995841,-5.077978,-0.597449,1.100032e-01,0.699547,1.996542
air_temp_min,14871.0,3.859359e-03,0.997920,-3.426343,-0.743237,-1.852529e-03,0.845444,2.451777
mean_cloud_coverage,14871.0,5.417200e-03,0.998010,-2.538051,-0.642811,1.626656e-01,0.825999,1.252428
temp_max,14871.0,1.041365e-03,1.000566,-3.059947,-0.787789,-1.926526e-02,0.832794,2.921174


In [17]:
weather.to_csv("../data/best_covs_longer.csv")

In [18]:
df = pd.read_csv("../../BBDC_2023/data/best_covs_longer.csv",index_col=0,parse_dates=True)
df

,precipitation_form,temp_min,snow_depth,sunshine_duration,air_pressure_mean,wind_spead_mean,relative_humidity,air_temp_min,mean_cloud_coverage,temp_max,precipitation_height,wind_max,vapor_pressure_mean,temp_mean,year,time_as_number,Year_sin,Year_cos,Weekday_sin,Weekday_cos
Zeitstempel,,,,,,,,,,,,,,,,,,,,
1960-01-01,-0.749456,-0.213730,-0.123224,-1.051992,-1.253528,0.147239,1.053273,-0.143069,0.304809,-0.470355,0.143169,0.167425,-0.254791,-0.370352,-1.701822,NaN,-0.013988,1.413990,-0.613490,-1.274064
1960-01-04,-0.749456,-0.775460,-0.123224,-1.051992,1.243440,-0.345040,1.760725,-0.584369,1.252428,-0.854617,-0.390230,-0.498596,-0.513871,-0.753596,-1.701822,NaN,0.058980,1.412829,0.000096,1.414265
1960-01-05,-0.749456,-0.354162,-0.123224,-1.051992,0.423972,0.854891,0.699547,-0.213677,0.304809,-0.520477,-0.365984,0.944450,-0.358423,-0.422612,-1.701822,NaN,0.083278,1.411606,1.105738,0.881809
1960-01-06,-0.749456,-0.441933,-0.123224,-0.249673,0.848167,0.854891,-0.243723,-0.284285,-0.311144,-0.637426,-0.414475,0.648440,-0.617503,-0.422612,-1.701822,NaN,0.107550,1.409964,1.378810,-0.314610
1960-01-07,-0.749456,-0.231284,-0.123224,-1.051992,0.462535,0.116472,1.053273,-0.196025,0.920761,-0.637426,-0.269003,0.074922,-0.358423,-0.457453,-1.701822,NaN,0.131791,1.407906,0.613681,-1.274064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-26,0.235459,-0.810568,-0.123224,-0.271960,0.327564,-0.002203,-0.715357,-0.690281,0.352190,-0.319992,-0.147776,-0.547931,-0.539779,-0.230991,1.701822,NaN,-0.131124,1.407968,0.000096,1.414265
2016-12-27,0.235459,-0.038189,-0.123224,-0.829126,1.879733,-0.116482,-0.479540,-0.125417,0.589094,-0.353406,-0.462966,-0.630156,-0.436147,-0.178730,1.701822,NaN,-0.106882,1.410015,1.105738,0.881809
2016-12-28,-1.077760,-0.512149,-0.123224,-1.051992,2.679920,-0.230761,-0.479540,-0.390197,0.446952,-0.553891,-0.487211,-0.712381,-0.643411,-0.405192,1.701822,NaN,-0.082608,1.411645,1.378810,-0.314610
